In [1]:
# !pip install -U git+https://github.com/openfisca/openfisca-ceq.git@master#egg=OpenFisca-CEQ
# !pip install -U git+https://github.com/openfisca/openfisca-senegal.git@ceq#egg=OpenFisca-Senegal
# !pip install -U git+https://github.com/openfisca/openfisca-mali.git@master#egg=OpenFisca-Mali
# !pip install -U git+https://github.com/openfisca/openfisca-cote-d-ivoire.git@master#egg=OpenFisca-COTE-D-IVOIRE

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from openfisca_ceq.tools.survey_scenario import build_ceq_survey_scenario
from openfisca_ceq.tools.indirect_taxation.tax_benefit_system_indirect_taxation_completion import indirect_tax_by_country
from openfisca_ceq.tools.data import year_by_country
from openfisca_ceq.tools.data.tax_targets import read_tax_target
from openfisca_ceq.tools.results.inequality import inequality_table, incidence_table, concentration_share, taxpayers_share
from openfisca_ceq.tools.data_ceq_correspondence import (
    ceq_input_by_harmonized_variable,
    ceq_intermediate_by_harmonized_variable,
    non_ceq_input_by_harmonized_variable,
    )
from openfisca_ceq.tools.tax_benefit_system_ceq_completion import labor_type_by_index

In [3]:
ceq_by_harmonized_variable = dict()
ceq_by_harmonized_variable.update(ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable.update(ceq_intermediate_by_harmonized_variable)
ceq_by_harmonized_variable.update(non_ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable['autres_revenus_du_capital_brut'] = 'autres_revenus_du_capital_brut'
ceq_by_harmonized_variable['pension_retraite_brut'] = 'pension_retraite_brut'
ceq_by_harmonized_variable['revenu_foncier_brut'] = 'revenu_foncier_brut'
ceq_by_harmonized_variable['revenu_non_salarie_brut'] = 'revenu_non_salarie_brut' 
ceq_by_harmonized_variable['salaire_brut'] = 'salaire_brut'
ceq_by_harmonized_variable['salaire_super_brut'] = 'salaire_super_brut'


In [4]:
def income_table(survey_scenario, inflated_survey_scenario):
    data = [
        (
            harmonized_variable, 
            openfisca_variable, 
            survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9,
            inflated_survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9,
            ) 
        for harmonized_variable, openfisca_variable in ceq_by_harmonized_variable.items()
        ]
    revenus = pd.DataFrame(data, columns = ["harmonized", "openfisca", "aggregate", "inflated aggregate"])
    
    return revenus.round(0).astype({"aggregate": int, "inflated aggregate": int})

In [5]:
def wage_earner(survey_scenario):
    df = pd.concat(
        [
            (
                survey_scenario.compute_pivot_table(
                    aggfunc = 'sum', 
                    values = ["salaire_brut", "salaire_super_brut"], 
                    index = "secteur_public", 
                    period = survey_scenario.year, 
                    concat_axis = 1
                    ) / 1e9
                ).round(0).astype(int)
            ],
        axis = 1,
        )
    df.loc[True, "effectifs"] = (
        (survey_scenario.calculate_variable('salaire_brut', period =survey_scenario.year) > 0) 
        * survey_scenario.calculate_variable('secteur_public', period =survey_scenario.year)
        * survey_scenario.calculate_variable('person_weight', period =survey_scenario.year)
        ).sum()
    df.loc[False, "effectifs"] = (
        (survey_scenario.calculate_variable('salaire_brut', period =survey_scenario.year) > 0) 
        * np.logical_not(survey_scenario.calculate_variable('secteur_public', period =survey_scenario.year))
        * survey_scenario.calculate_variable('person_weight', period = survey_scenario.year)
        ).sum()
    return df.round(0).astype(int)

In [6]:
def income_distribution(survey_scenario):
    survey_income = (
        survey_scenario.compute_pivot_table(
            values = "survey_income", columns = "decile_gross_income_per_capita", period = survey_scenario.year
            ).transpose().round()    
        )
    gross_income = (
        survey_scenario.compute_pivot_table(
            values = "gross_income", columns = "decile_gross_income_per_capita", period = survey_scenario.year
            ).transpose().round()
        )
    return pd.concat([survey_income, gross_income], axis = 1)

## Inequality tables

In [7]:
for country, year in year_by_country.items():
    survey_scenario = build_ceq_survey_scenario(country, year)
    inflated_survey_scenario = build_ceq_survey_scenario(country, year, inflate = True)
    display(country)
    display(income_table(survey_scenario, inflated_survey_scenario))
    display(wage_earner(survey_scenario))

/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  im

'cote_d_ivoire'

,harmonized,openfisca,aggregate,inflated aggregate
0,rev_i_autoconsommation,autoconsumption,1289,1887
1,rev_i_autres,other_income,581,851
2,rev_i_autres_transferts,gifts_sales_durables,782,1144
3,rev_i_loyers_imputes,imputed_rent,277,406
4,rev_i_transferts_publics,direct_transfers,8,8
5,rev_i_agricoles,revenu_agricole,1428,2091
6,rev_i_autres_revenus_capital,autres_revenus_du_capital,75,75
7,rev_i_independants_Ntaxe,revenu_informel_non_salarie,1346,1970
8,rev_i_independants_taxe,revenu_non_salarie,245,245
9,rev_i_locatifs,revenu_locatif,215,215


,salaire_brut,salaire_super_brut,effectifs
secteur_public,,,
False,781,862,238884
True,835,925,219514


'mali'

...

,salaire_brut,salaire_super_brut,effectifs
secteur_public,,,
False,48,57,89865
True,33,39,71994


/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


'senegal'

,harmonized,openfisca,aggregate,inflated aggregate
0,rev_i_autoconsommation,autoconsumption,158,219
1,rev_i_autres,other_income,149,206
2,rev_i_autres_transferts,gifts_sales_durables,714,992
3,rev_i_loyers_imputes,imputed_rent,364,505
4,rev_i_transferts_publics,direct_transfers,24,24
5,rev_i_agricoles,revenu_agricole,1062,1475
6,rev_i_autres_revenus_capital,autres_revenus_du_capital,13,13
7,rev_i_independants_Ntaxe,revenu_informel_non_salarie,866,1203
8,rev_i_independants_taxe,revenu_non_salarie,263,263
9,rev_i_locatifs,revenu_locatif,54,54


,salaire_brut,salaire_super_brut,effectifs
secteur_public,,,
False,464,500,123901
True,412,441,84224


In [8]:
income_variable = 'gross_income'
by_variable = 'decile_{}_per_capita'.format(income_variable)
tax_variables = ['personal_income_tax', 'customs_duties', 'value_added_tax', 'education_net_transfers']

table_by_indicator_by_scenario = dict()

for survey_type in ["bare" , "inflated"]:
    survey_scenario = build_ceq_survey_scenario(country, year, inflate = survey_type == "inflated")
    income_by_country = dict()
    gini_by_country = dict()
    incidence_by_country = dict()
    concentration_share_by_country = dict()
    taxpayers_share_by_country = dict()
    for country, year in year_by_country.items():
        survey_scenario = build_ceq_survey_scenario(country, year)
        income_by_country[country] = income_distribution(survey_scenario)
        gini_by_country[country] = inequality_table(survey_scenario)
        incidence_by_country[country] = incidence_table(survey_scenario, income_variable, tax_variables, by_variable)
        concentration_share_by_country[country] = concentration_share(survey_scenario, tax_variables, by_variable)  
        taxpayers_share_by_country[country] = taxpayers_share(survey_scenario, tax_variables, by_variable)
        display('incidence - ' + country + ' - ' + survey_type)
        display(incidence_table(survey_scenario, income_variable, tax_variables, by_variable = 'labor_type').rename(index = labor_type_by_index))
        display('concentration_share - ' + country + ' - ' + survey_type)
        display(concentration_share(survey_scenario, tax_variables, by_variable = 'labor_type').rename(index = labor_type_by_index))
        display('taxpayers_share - ' + country + ' - ' + survey_type)
        display(taxpayers_share(survey_scenario, tax_variables, by_variable = 'labor_type').rename(index = labor_type_by_index))
        table_by_indicator = dict(
            ginis = pd.concat(gini_by_country).swaplevel().sort_index(),
            incidences = pd.concat(incidence_by_country).transpose().stack(),
            concentrations = pd.concat(concentration_share_by_country).transpose().stack(),
            taxpayers = pd.concat(taxpayers_share_by_country).transpose().stack(),
            )
        table_by_indicator_by_scenario[survey_type] = table_by_indicator

/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


'incidence - cote_d_ivoire - bare'

,personal_income_tax,customs_duties,value_added_tax,education_net_transfers
labor_type,,,,
Inactive,0.01,0.03,0.08,0.15
Formal public wage worker,0.08,0.02,0.05,0.07
Formal private wage worker,0.08,0.02,0.05,0.04
Informal wage worker,0.00,0.02,0.07,0.08
Informal independent worker,0.00,0.02,0.07,0.07
Agricultural worker,0.00,0.01,0.05,0.06


'concentration_share - cote_d_ivoire - bare'

,personal_income_tax,customs_duties,value_added_tax,education_net_transfers
labor_type,,,,
Inactive,0.11,0.24,0.23,0.34
Formal public wage worker,0.40,0.07,0.07,0.07
Formal private wage worker,0.42,0.07,0.06,0.04
Informal wage worker,0.05,0.28,0.28,0.23
Informal independent worker,0.01,0.18,0.18,0.14
Agricultural worker,0.01,0.16,0.18,0.17


...

,personal_income_tax,customs_duties,value_added_tax,education_net_transfers
labor_type,,,,
Inactive,0.03,0.02,0.07,0.09
Formal public wage worker,0.18,0.01,0.05,0.05
Formal private wage worker,0.14,0.01,0.05,0.05
Informal wage worker,0.01,0.02,0.07,0.09
Informal independent worker,0.01,0.02,0.06,0.05
Agricultural worker,0.00,0.02,0.05,0.10


'concentration_share - senegal - inflated'

,personal_income_tax,customs_duties,value_added_tax,education_net_transfers
labor_type,,,,
Inactive,0.28,0.38,0.38,0.38
Formal public wage worker,0.36,0.04,0.05,0.05
Formal private wage worker,0.28,0.04,0.06,0.04
Informal wage worker,0.02,0.09,0.10,0.09
Informal independent worker,0.04,0.21,0.22,0.15
Agricultural worker,0.03,0.24,0.19,0.29


'taxpayers_share - senegal - inflated'

,personal_income_tax,customs_duties,value_added_tax,education_net_transfers
labor_type,,,,
Inactive,0.18,0.99,1.0,0.61
Formal public wage worker,0.99,0.99,1.0,0.58
Formal private wage worker,0.99,1.00,1.0,0.49
Informal wage worker,0.07,1.00,1.0,0.58
Informal independent worker,0.10,1.00,1.0,0.50
Agricultural worker,0.03,0.99,1.0,0.59


In [9]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [10]:
sheet_name_by_indicator = dict(
    ginis = "r1",
    incidences = "r2",
    concentrations = "r3",
    taxpayers = "r4",
    )
for survey_type, table_by_indicator in table_by_indicator_by_scenario.items():
    with pd.ExcelWriter(
        '/home/benjello/Dropbox/Projet_Micro_Sim/resultats/graphs-tableaux/Resultats_Taxineq_{}.xlsx'.format(survey_type)
        ) as writer:
            for indicator, table in table_by_indicator.items(): 
                display(indicator + " - " + survey_type)
                display(table)
                table.to_excel(writer, sheet_name = sheet_name_by_indicator[indicator])

'ginis - bare'

market_income  market_income_plus_pensions  \
Bottom 40 % cote_d_ivoire           0.09                         0.09   
            mali                    0.13                         0.13   
            senegal                 0.10                         0.10   
Gini        cote_d_ivoire           0.58                         0.57   
            mali                    0.46                         0.46   
            senegal                 0.56                         0.56   
Top 10 %    cote_d_ivoire           0.46                         0.46   
            mali                    0.35                         0.35   
            senegal                 0.44                         0.44   

                           gross_income  disposable_income  consumable_income  \
Bottom 40 % cote_d_ivoire          0.09               0.09               0.08   
            mali                   0.13               0.13               0.09   
            senegal                0.10               0.10               0.09   
Gini        cote_d_ivoire          0.57               0.57               0.60   
            mali                   0.46               0.46               0.53   
            senegal                0.55               0.54               0.56   
Top 10 %    cote_d_ivoire          0.46               0.45               0.46   
            mali                   0.35               0.35               0.38   
            senegal                0.44               0.42               0.44   

                           final_income  
Bottom 40 % cote_d_ivoire          0.10  
            mali                   0.13  
            senegal                0.11  
Gini        cote_d_ivoire          0.56  
            mali                   0.46  
            senegal                0.53  
Top 10 %    cote_d_ivoire          0.44  
            mali                   0.34  
            senegal                0.42

'incidences - bare'

cote_d_ivoire  mali  \
                        decile_gross_income_per_capita                        
personal_income_tax     1                                        0.00  0.00   
                        2                                        0.00  0.00   
                        3                                        0.00  0.00   
                        4                                        0.00  0.00   
                        5                                        0.00  0.00   
                        6                                        0.00  0.00   
                        7                                        0.00  0.00   
                        8                                        0.00  0.00   
                        9                                        0.01  0.00   
                        10                                       0.03  0.01   
customs_duties          1                                        0.18  0.11   
                        2                                        0.06  0.04   
                        3                                        0.04  0.04   
                        4                                        0.04  0.02   
                        5                                        0.03  0.02   
                        6                                        0.03  0.02   
                        7                                        0.02  0.01   
                        8                                        0.02  0.01   
                        9                                        0.02  0.01   
                        10                                       0.01  0.01   
value_added_tax         1                                        0.50  0.88   
                        2                                        0.18  0.27   
                        3                                        0.13  0.22   
                        4                                        0.11  0.17   
                        5                                        0.10  0.15   
                        6                                        0.09  0.12   
                        7                                        0.08  0.11   
                        8                                        0.06  0.10   
                        9                                        0.06  0.08   
                        10                                       0.04  0.05   
education_net_transfers 1                                        1.12  1.23   
                        2                                        0.40  0.46   
                        3                                        0.30  0.34   
                        4                                        0.21  0.22   
                        5                                        0.16  0.21   
                        6                                        0.16  0.16   
                        7                                        0.09  0.14   
                        8                                        0.08  0.11   
                        9                                        0.06  0.08   
                        10                                       0.02  0.04   

                                                        senegal  
                        decile_gross_income_per_capita           
personal_income_tax     1                                  0.00  
                        2                                  0.00  
                        3                                  0.00  
                        4                                  0.00  
                        5                                  0.00  
                        6                                  0.00  
                        7                                  0.01  
                        8                                  0.01  
                        9                                  0.02  
           

'concentrations - bare'

...

cote_d_ivoire  mali  \
                        decile_gross_income_per_capita                        
personal_income_tax     1                                        0.00  0.00   
                        2                                        0.00  0.00   
                        3                                        0.00  0.00   
                        4                                        0.00  0.00   
                        5                                        0.00  0.00   
                        6                                        0.00  0.00   
                        7                                        0.00  0.00   
                        8                                        0.00  0.00   
                        9                                        0.01  0.00   
                        10                                       0.03  0.01   
customs_duties          1                                        0.18  0.11   
                        2                                        0.06  0.04   
                        3                                        0.04  0.04   
                        4                                        0.04  0.02   
                        5                                        0.03  0.02   
                        6                                        0.03  0.02   
                        7                                        0.02  0.01   
                        8                                        0.02  0.01   
                        9                                        0.02  0.01   
                        10                                       0.01  0.01   
value_added_tax         1                                        0.50  0.88   
                        2                                        0.18  0.27   
                        3                                        0.13  0.22   
                        4                                        0.11  0.17   
                        5                                        0.10  0.15   
                        6                                        0.09  0.12   
                        7                                        0.08  0.11   
                        8                                        0.06  0.10   
                        9                                        0.06  0.08   
                        10                                       0.04  0.05   
education_net_transfers 1                                        1.12  1.23   
                        2                                        0.40  0.46   
                        3                                        0.30  0.34   
                        4                                        0.21  0.22   
                        5                                        0.16  0.21   
                        6                                        0.16  0.16   
                        7                                        0.09  0.14   
                        8                                        0.08  0.11   
                        9                                        0.06  0.08   
                        10                                       0.02  0.04   

                                                        senegal  
                        decile_gross_income_per_capita           
personal_income_tax     1                                  0.00  
                        2                                  0.00  
                        3                                  0.00  
                        4                                  0.00  
                        5                                  0.00  
                        6                                  0.00  
                        7                                  0.01  
                        8                                  0.01  
                        9                                  0.02  
           

'concentrations - inflated'

cote_d_ivoire  mali  \
                        decile_gross_income_per_capita                        
personal_income_tax     1                                        0.00  0.00   
                        2                                        0.00  0.00   
                        3                                        0.00  0.00   
                        4                                        0.00  0.00   
                        5                                        0.00  0.01   
                        6                                        0.00  0.01   
                        7                                        0.01  0.03   
                        8                                        0.03  0.03   
                        9                                        0.09  0.06   
                        10                                       0.86  0.86   
customs_duties          1                                        0.06  0.09   
                        2                                        0.06  0.07   
                        3                                        0.06  0.10   
                        4                                        0.07  0.07   
                        5                                        0.08  0.08   
                        6                                        0.08  0.09   
                        7                                        0.10  0.10   
                        8                                        0.11  0.11   
                        9                                        0.14  0.13   
                        10                                       0.26  0.16   
value_added_tax         1                                        0.05  0.11   
                        2                                        0.05  0.07   
                        3                                        0.05  0.08   
                        4                                        0.06  0.08   
                        5                                        0.07  0.09   
                        6                                        0.08  0.09   
                        7                                        0.09  0.10   
                        8                                        0.11  0.11   
                        9                                        0.15  0.13   
                        10                                       0.28  0.16   
education_net_transfers 1                                        0.09  0.12   
                        2                                        0.09  0.10   
                        3                                        0.10  0.10   
                        4                                        0.09  0.08   
                        5                                        0.09  0.10   
                        6                                        0.12  0.09   
                        7                                        0.08  0.10   
                        8                                        0.11  0.10   
                        9                                        0.12  0.09   
                        10                                       0.10  0.10   

                                                        senegal  
                        decile_gross_income_per_capita           
personal_income_tax     1                                  0.00  
                        2                                  0.00  
                        3                                  0.00  
                        4                                  0.00  
                        5                                  0.00  
                        6                                  0.01  
                        7                                  0.01  
                        8                                  0.04  
                        9                                  0.09  
           

'taxpayers - inflated'

cote_d_ivoire  mali  \
                        decile_gross_income_per_capita                        
personal_income_tax     1                                        0.02  0.01   
                        2                                        0.04  0.00   
                        3                                        0.04  0.03   
                        4                                        0.05  0.03   
                        5                                        0.05  0.11   
                        6                                        0.06  0.04   
                        7                                        0.08  0.05   
                        8                                        0.13  0.05   
                        9                                        0.16  0.07   
                        10                                       0.27  0.11   
customs_duties          1                                        0.99  0.97   
                        2                                        1.00  1.00   
                        3                                        0.99  0.99   
                        4                                        1.00  1.00   
                        5                                        0.99  1.00   
                        6                                        1.00  0.99   
                        7                                        1.00  1.00   
                        8                                        1.00  1.00   
                        9                                        1.00  1.00   
                        10                                       1.00  0.99   
value_added_tax         1                                        1.00  0.97   
                        2                                        1.00  1.00   
                        3                                        1.00  0.99   
                        4                                        1.00  1.00   
                        5                                        1.00  1.00   
                        6                                        1.00  0.99   
                        7                                        1.00  1.00   
                        8                                        1.00  1.00   
                        9                                        1.00  1.00   
                        10                                       1.00  0.99   
education_net_transfers 1                                        0.37  0.70   
                        2                                        0.43  0.73   
                        3                                        0.46  0.67   
                        4                                        0.39  0.63   
                        5                                        0.37  0.70   
                        6                                        0.35  0.63   
                        7                                        0.31  0.66   
                        8                                        0.32  0.63   
                        9                                        0.25  0.63   
                        10                                       0.15  0.53   

                                                        senegal  
                        decile_gross_income_per_capita           
personal_income_tax     1                                  0.01  
                        2                                  0.02  
                        3                                  0.02  
                        4                                  0.02  
                        5                                  0.03  
                        6                                  0.08  
                        7                                  0.12  
                        8                                  0.25  
                        9                                  0.32  
           

In [11]:
result = read_tax_target(add_country_details=True)
display(result)

/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/home/benjello/.virtualenvs/ceq/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  im

Cote d'Ivoire          \
                                                       actual  direct   
Total tax revenue                                      3117.9     NaN   
Direct taxes                                           1186.8   178.9   
Personal Income Taxes                                   395.6   178.9   
Social Security Contributions                           383.5     NaN   
Corporate Income Tax                                    318.9     0.0   
Other Direct Taxes                                       88.8     NaN   
Indirect taxes                                         1931.1   890.7   
VAT                                                     732.5   682.7   
Import Taxes                                            490.1   208.0   
Excise taxes                                             33.1     0.0   
on Oil Derivates                                          0.1     NaN   
on alcohol, tabac and other non-oil derivatives          33.0     NaN   
Other Indirect Taxes                                    675.4     NaN   
Share simulated taxes (% of total tax revenue)            NaN     NaN   
impot_general_revenu                                      NaN   155.8   
impot_revenu_creances                                     NaN    16.4   
impot_revenu_foncier                                      NaN     6.7   
total_ht_imports                                          NaN  1428.8   
impot_traitement_salaire                                  NaN     NaN   
contribution_globale_fonciere                             NaN     NaN   
contribution_globale_unique                               NaN     NaN   
droit_progressif_pension_retraite                         NaN     NaN   
droit_progressif_salaire                                  NaN     NaN   
droit_progressif                                          NaN     NaN   
droit_proportionnel_autres_revenus                        NaN     NaN   
droit_proportionnel_salaire                               NaN     NaN   
droit_proportionnel                                       NaN     NaN   
impot_revenus                                             NaN     NaN   

                                                           Mali         \
                                                inflated actual direct   
Total tax revenue                                    NaN  873.9    NaN   
Direct taxes                                       326.9  307.5    7.4   
Personal Income Taxes                              326.9   57.6    7.4   
Social Security Contributions                        NaN   86.7    NaN   
Corporate Income Tax                                 0.0  113.3    0.0   
Other Direct Taxes                                   NaN   49.9    NaN   
Indirect taxes                                     961.1  566.4  267.3   
VAT                                                736.7  282.2  235.4   
Import Taxes                                       224.4  112.1   31.9   
Excise taxes                                         0.0   16.1    0.0   
on Oil Derivates                                     NaN    4.7    NaN   
on alcohol, tabac and other non-oil derivatives      NaN   11.4    NaN   
Other Indirect Taxes                                 NaN  156.0    NaN   
Share simulated taxes (% of total tax revenue)       NaN    NaN    NaN   
impot_general_revenu                               293.2    NaN    NaN   
impot_revenu_creances                               24.0    NaN    NaN   
impot_revenu_foncier                                 9.7    NaN    NaN   
total_ht_imports                                  1541.8    NaN  418.6   
impot_traitement_salaire                             NaN    NaN    7.4   
contribution_globale_fonciere                        NaN    NaN    NaN   
contribution_globale_unique                          NaN    NaN    NaN   
droit_progressif_pension_retraite                    NaN    NaN    NaN   
droit_progressif_salaire                             NaN    NaN    NaN   
droit_p